## Ring resonator using a local implicit time stepping

In [ ]:
import math
from ngsolve import *
from ngsolve.webgui import Draw
import netgen.geom2d as gm
import numpy as np
from tqdm import tqdm
SetHeapSize(50*1000*1000)
# geometrical and material properties

In [ ]:
geo = gm.SplineGeometry()

xneg  =-0.43#-0.1
xpos  = 0.43#+0.1
yneg  =-0.48#-0.1
ypos  = 0.48#+0.1
wslab = 0.04
cringx = 0.0
cringy = 0.0
rring = 0.4
gap   = 0.005

In [ ]:
pntx = [xneg,xpos]
pnty = [yneg,-rring-gap-wslab,-rring-gap,rring+gap,rring+gap+wslab,ypos]


pts = []
for yi in pnty:
    for xi in pntx:
        pts.append (geo.AddPoint(xi,yi))

#### parameters for source position


### geometry #######################################################
#inner rects
geo.Append (["line", pts[0], pts[1]], leftdomain=1, rightdomain=0)
geo.Append (["line", pts[1], pts[3]], leftdomain=1, rightdomain=0)
geo.Append (["line", pts[3], pts[2]], leftdomain=1, rightdomain=2)
geo.Append (["line", pts[2], pts[0]], leftdomain=1, rightdomain=0)

geo.Append (["line", pts[3], pts[5]], leftdomain=2, rightdomain=0,bc="normal_wg_rightbottom")
geo.Append (["line", pts[5], pts[4]], leftdomain=2, rightdomain=3)
geo.Append (["line", pts[4], pts[2]], leftdomain=2, rightdomain=0,bc="normal_wg_leftbottom")

geo.Append (["line", pts[5], pts[7]], leftdomain=3, rightdomain=0)
geo.Append (["line", pts[7], pts[6]], leftdomain=3, rightdomain=4)
geo.Append (["line", pts[6], pts[4]], leftdomain=3, rightdomain=0)

geo.Append (["line", pts[7], pts[9]], leftdomain=4, rightdomain=0,bc="normal_wg_righttop")
geo.Append (["line", pts[9], pts[8]], leftdomain=4, rightdomain=5)
geo.Append (["line", pts[8], pts[6]], leftdomain=4, rightdomain=0,bc="normal_wg_lefttop")

geo.Append (["line", pts[9], pts[11]], leftdomain=5, rightdomain=0)
geo.Append (["line", pts[11], pts[10]], leftdomain=5, rightdomain=0)
geo.Append (["line", pts[10], pts[8]], leftdomain=5, rightdomain=0)

geo.AddCircle(c=(cringx,cringy), r=rring, leftdomain=6, rightdomain=3)
geo.AddCircle(c=(cringx,cringy), r=rring-wslab, leftdomain=7, rightdomain=6)

for i in (1,3,5,7):
    geo.SetMaterial(i, "air")
for i in (2,4,6):
    geo.SetMaterial(i, "eps_nine")
data = geo.CreatePML(0.05)
normals = data["normals"]

h = 0.05
mesh = Mesh(geo.GenerateMesh(maxh=h))

Draw(mesh)

In [ ]:
eps_r = {"air" : 1,
         "eps_nine" : 3**3}

order = 4

for mat in mesh.GetMaterials():
    if mat.startswith("pml_normal_wg"):
        eps_r[mat] = eps_r["eps_nine"]

for mat in mesh.GetMaterials():
    if mat not in eps_r:
        eps_r[mat] = eps_r["air"]

print ('materials:',[mat for mat in mesh.GetMaterials()])

def gaussp(pt):
    return CoefficientFunction(sin( (math.pi/wslab)*(y-pt[1])))
    #return CoefficientFunction(exp ( -3200 * ( (y-pt[1])**2 )) )

# ~ center_source = (pntx[0],0.5*pnty[3]+0.5*pnty[4])
center_source = (pntx[0],pnty[3])
print("\neps_r:", eps_r)

In [ ]:
### Parameters for Source field ##########################################################################
wavelength = 1.542
fcen       = 5/wavelength
df         = 0.1
tpeak      = 1

fes_facet = FacetFESpace(mesh, order=order+1)
gfsource = GridFunction(fes_facet)
source_cf = gaussp(center_source)
gfsource.Set(source_cf,definedon=mesh.Boundaries("normal_wg_lefttop"))

fes_test = H1(mesh)
gf_test = GridFunction(fes_test)
gf_test.Set(source_cf)
Draw(gf_test)

In [ ]:
tau = 1e-3 #was 2e-4
dt = tau
fes_u = VectorL2(mesh, order=order, piola=True, order_policy=ORDER_POLICY.CONSTANT, dgjumps = True)
fes_p = L2(mesh, order=order+1, all_dofs_together=True, order_policy=ORDER_POLICY.CONSTANT, dgjumps = True)
fes_tr = FacetFESpace(mesh, order=order+1)
fes_hdiv = HDiv(mesh, order=order+1, orderinner=1)
fes = fes_p*fes_p*fes_u*fes_u

p,phat, u,uhat = fes.TrialFunction()
q,qhat, v,vhat = fes.TestFunction()

n = specialcf.normal(2) 
h = specialcf.mesh_size

In [ ]:
print('mesh elements:', mesh.ne)
print('fes ndof:', fes.ndof)

In [ ]:
def get_implict_dofs(mesh, fes, fes_u):
    F = specialcf.JacobianMatrix(mesh.dim)
    Finv = Inv(F)
    detF = Det(F)
    Norm_Finv = Norm(Finv)
    
    el_norms = Integrate(Norm_Finv*1/detF.Norm(), mesh, element_wise=True)
    el_norms_numpy = np.array(el_norms)

    sorted_el_norms = -np.sort(-el_norms_numpy) #sort descending
    ref_index = int(0.55*sqrt(len(sorted_el_norms))) # sqrt(n) elements as implcit where n is mesh.ne,
    ref_norm = sorted_el_norms[ref_index]
    #ref_norm = el_norms_numpy.mean()# using elements > mean seams best
    impl_els = np.where(el_norms_numpy > ref_norm, 1, 0)
    
    ba_implicit_els = BitArray(mesh.ne)
    ba_explicit_els = BitArray(mesh.ne)
    ba_interface_edges = BitArray(mesh.nedge)
    ba_explicit_edges = BitArray(mesh.nedge)
    ba_implicit_edges = BitArray(mesh.nedge)

    ba_implicit_els[:] = 0
    ba_explicit_els[:] = 0
    ba_interface_edges[:] = 0
    ba_explicit_edges[:] = 0
    ba_implicit_edges[:] = 0

    for el in mesh.Elements():
        if impl_els[el.nr] == 1:
            ba_implicit_els[el.nr] = 1
            for e in el.edges:
                ba_implicit_edges[e.nr] = 1
        else:
            ba_explicit_els[el.nr] = 1
            for e in el.edges:
                ba_explicit_edges[e.nr] = 1

    ba_interface_edges = ba_explicit_edges & ba_implicit_edges     
    ba_local_implicit_dofs = BitArray(fes.ndof)
    ba_local_implicit_dofs[:] = 0

    for el in mesh.Elements():
        if ba_implicit_els[el.nr] == 1:
            for nr in fes.GetDofNrs(el):
                ba_local_implicit_dofs[nr] = 1
        if ba_explicit_els[el.nr] == 1:
            for e in el.edges:
                if ba_interface_edges[e.nr] == 1:
                    for nr in fes.GetDofNrs(el):
                        ba_local_implicit_dofs[nr] = 1

    ba_local_implicit_dofs_u = BitArray(fes_u.ndof)
    ba_local_implicit_dofs_u[:] = 0

    for el in mesh.Elements():
        if ba_implicit_els[el.nr] == 1:
            for nr in fes_u.GetDofNrs(el):
                ba_local_implicit_dofs_u[nr] = 1
        if ba_explicit_els[el.nr] == 1:
            for e in el.edges:
                if ba_interface_edges[e.nr] == 1:
                    for nr in fes_u.GetDofNrs(el):
                        ba_local_implicit_dofs_u[nr] = 1
    return ba_local_implicit_dofs, ba_local_implicit_dofs_u

In [ ]:
u_p_dofs = BitArray([*[x for x in fes_p.FreeDofs()], *[False for _ in (fes.Range(1))], *[x for x in fes_u.FreeDofs()], *[False for _ in (fes.Range(3))]])
ba_local_implicit_dofs, ba_local_implicit_dofs_u = get_implict_dofs(mesh, fes, fes_u)
ba_local_implicit_dofs = ba_local_implicit_dofs & u_p_dofs

In [ ]:
print ("local implicit dofs: ", ba_local_implicit_dofs.NumSet(),"/",len(ba_local_implicit_dofs))
print ("local implicit dofs of fes u: ", ba_local_implicit_dofs_u.NumSet(),"/",len(ba_local_implicit_dofs_u))

In [ ]:
# determine implict elements, only for testing
ba_gfu = BitArray(fes.ndof)
ba_gfu[:] = 0
gfuim = GridFunction(fes)
gfuim.vec[:] = 0
for i in range(len(ba_local_implicit_dofs)):
    if ba_local_implicit_dofs[i] == 1:
        gfuim.vec.data[i] = 1
        
scene1 = Draw(gfuim.components[0], mesh)
#scene2 = Draw(gfuim.components[2], mesh)

In [ ]:
Bel = BilinearForm(trialspace=fes_p, testspace=fes_u, geom_free=True)
Bel += ( grad(fes_p.TrialFunction())*fes_u.TestFunction()) * dx
Bel += ( -fes_p.TrialFunction()*(fes_u.TestFunction()*n))* dx(element_boundary=True)
Bel.Assemble()

Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u, geom_free=True)
Btr += (0.5 * fes_tr.TrialFunction() * (n*fes_u.TestFunction())) * dx(element_boundary=True)
Btr.Assemble()

Bstab = BilinearForm(trialspace=fes_p, testspace=fes_hdiv, geom_free=True)
Bstab += (fes_p.TrialFunction()*(fes_hdiv.TestFunction()*n)) * dx(element_boundary=True)
Bstab.Assemble()

Mstab = BilinearForm(fes_hdiv)
Mstab += (fes_hdiv.TrialFunction() * fes_hdiv.TestFunction()) * dx
Mstab.Assemble()
Mstabinv = Mstab.mat.CreateSmoother()

In [ ]:
### LinearForm for the source ##############################################
Lsrc  = LinearForm(fes)
Lsrc  += (gfsource*q) * dx(element_boundary=True)
Lsrc.Assemble()

In [ ]:
nvec = { mat : ((normals[mat][0], normals[mat][1]) if mat in normals else (0,0)) for mat in mesh.GetMaterials() }

cfn = CoefficientFunction( [CoefficientFunction(nvec[mat]) for mat in mesh.GetMaterials()])
cft = CoefficientFunction( ( cfn[1], -cfn[0] ) )

pml1d = mesh.Materials("pml_default.*|pml_normal.*")

print(pml1d.Mask())

eps = CoefficientFunction([eps_r[mat] for mat in mesh.GetMaterials()])

Draw(eps, mesh, "eps")

In [ ]:
# damping matrices
sigma = 10   # pml damping parameter
dampp1 = fes_p.Mass(eps, definedon=pml1d)
dampp2 = fes_p.Mass(eps, definedon=mesh.Materials("pml_corner"))
dampu1 = fes_u.Mass(OuterProduct(cfn,cfn), definedon=pml1d)
dampu2 = fes_u.Mass(OuterProduct(cft,cft), definedon=pml1d)

gfu = GridFunction(fes)
gfu_half_step = GridFunction(fes)
gfu.vec[:] = 0
gfu_half_step.vec[:] = 0
# gfu.components[0].Set (gaussp(prism1_center) + gaussp(mirrory(prism1_center)))

scalval = 1e-1
Draw(gfu.components[0], mesh, "p")

In [ ]:
w = gfu.vec.CreateVector()
gfuold = gfu.vec.CreateVector()
res = gfu.vec.CreateVector()
#hvtrace = BaseVector(fes_tr.ndof)
#hvu = BaseVector(fes_u.ndof)
#hvu2 = BaseVector(fes_u.ndof)

emb_p, emb_phat, emb_u, emb_uhat = fes.embeddings

traceop = fes_p.TraceOperator(fes_tr, False)
fullB = emb_u @ (Bel.mat + Btr.mat @ traceop) @ emb_p.T
B = Bel.mat + Btr.mat @ traceop
dampingu = emb_u @ dampu1 @ emb_u.T + (-emb_u + emb_uhat) @ dampu2 @ (emb_u.T + emb_uhat.T)
dampingp = emb_p @ dampp1 @ emb_p.T + emb_p @ dampp2 @ (2*emb_p.T-emb_phat.T) + emb_phat @ dampp2 @ emb_p.T

# not needed anymore
du = emb_u @ (dampu1 - dampu2) @ emb_u.T - emb_u @ dampu2 @ emb_uhat.T
dupml = emb_uhat @ dampu2 @ emb_u.T + emb_uhat @ dampu2 @ emb_uhat.T

invmassp = fes_p.Mass(eps).Inverse()
invmassu = fes_u.Mass(Id(mesh.dim)).Inverse()
invp = emb_p @ invmassp @ emb_p.T + emb_phat @ invmassp @ emb_phat.T
invu = emb_u @ invmassu @ emb_u.T
invupml = emb_uhat @ invmassu @ emb_uhat.T

In [ ]:
# for local implicit
dS = dx(skeleton=True)   
def jump(p): return p.Other()-p
def avgn(v): return 0.5*(v*n-v.Other()*n.Other())

A = BilinearForm(fes)
A += -eps*p*q*dx +u*v*dx 
A += dt/2*(grad(p)*v + grad(q)*u)*dx
A += dt/2*(jump(p)*avgn(v)+jump(q)*avgn(u)) * dS
A.Assemble()
Anze = A.mat.DeleteZeroElements(10e-12)
invAnze = Anze.Inverse(freedofs=ba_local_implicit_dofs, inverse="sparsecholesky")
invmstar = emb_u.T @ invAnze @ emb_u

In [ ]:
Ps = Projector(ba_local_implicit_dofs_u, True)   # projection to small
Pl = Projector(ba_local_implicit_dofs_u, False)  # projection to large
B_e = Pl @ B 
B_i = Ps @ B
mass_u = fes_u.Mass(Id(mesh.dim))
invmass_p = fes_p.Mass(eps).Inverse()
mstarloc = mass_u + dt*dt/4*B_i @ invmass_p @ B.T
invmstar_mstar = invmstar @ mstarloc
print ("local implicit dofs: ", ba_local_implicit_dofs.NumSet(),"/",len(ba_local_implicit_dofs))
print ("local implicit dofs of fes u: ", ba_local_implicit_dofs_u.NumSet(),"/",len(ba_local_implicit_dofs_u))
print ("shape B_e: ", B_e.shape)
print ("shape B_i: ", B_i.shape)
print ("shape Ps: ", Ps.shape)
print ("invmstar shape:", invmstar.shape)
print ("dt:", dt)

In [ ]:
fullB_e = emb_u @ B_e @ emb_p.T
fullB_i = emb_u @ B_i @ emb_p.T
fullinvmstar = emb_u @ invmstar @ emb_u.T 
fullmstarloc = emb_u @ mstarloc @ emb_u.T 
fullmassu = emb_u @ mass_u @ emb_u.T

In [ ]:
gfstab = GridFunction(fes_hdiv)
hvstab = gfstab.vec.CreateVector()

In [ ]:
# For the time stepping
tend = 25
t = 0
i = 0
n = 20

In [ ]:
scene = Draw (gfu.components[0], mesh, 'p', order=3, min=-0.05, max=0.05, autoscale=False)
loop = tqdm([i*tau for i in range(int(tend/tau))])

with TaskManager():#pajetrace=100*1000*1000):
    for t in loop:
        ### time envelope for the src ################################################################
        if abs((t-tpeak)/tpeak) < 1:
           t_envelope = (2*exp(1)/sqrt(math.pi))*sin(2*math.pi*fcen*t)*exp (-1/(1-((t-tpeak)/tpeak)**2))
        else:
           t_envelope = 0
        
        w.data = -fullB.T * gfu.vec
        w.data += t_envelope*Lsrc.vec
        w.data -= sigma * dampingp * gfu.vec

        gfu_half_step.vec.data = gfu.vec + tau/2 * invp * w
        
        w.data = tau * fullB_e * gfu_half_step.vec 
        w.data += tau/2 * fullB_i * (gfu_half_step.vec + gfu.vec) 
        w.data += (fullmassu - tau * du) * gfu.vec
        w.data += tau ** 2 /4 * fullB_i @ dampingp * gfu_half_step.vec
        gfuold.data = invu * w
        gfu.vec.data = gfuold 
        gfu.vec.data += fullinvmstar * (w - fullmstarloc * gfuold) 
        gfu.vec.data += emb_uhat * (emb_uhat.T * gfu_half_step.vec) + emb_p * (emb_p.T * gfu_half_step.vec) + emb_phat * (emb_phat.T * gfu_half_step.vec)
        
        w.data = - sigma * dupml * gfu.vec
        gfu.vec.data = gfu.vec + tau * invupml * w
        
        w.data = -fullB.T * gfu.vec
        w.data += t_envelope*Lsrc.vec
        w.data -= sigma * dampingp * gfu.vec
        
        gfu.vec.data = gfu.vec + tau/2 * invp * w 

        t += tau
        i+=1
        if i%n == 0:
            scene.Redraw()
        loop.set_description(f'Time step {t/tau:.0f}')
        loop.set_postfix_str(f'Time {t:.4f} seconds')